In [1]:
# run notebook in anaconda env named tf_notebook
# python -m ipykernel install --user --name=tf_notebook

import tensorflow as tf # neural network
import numpy as np # number processing
import nltk

import time# timing code
import os

Steps to generate text
1. Get data
2. Data preprocesssing
3. Making the dataset, set up input and output of model
4. set up model
5. train

# 1. Get data


In [2]:
# Getting file names

folder_path = "C:/Users/nthnt/PycharmProjects/TextGenerator/Datasets/Bionicle_Stories/"
# get files ending with txt = get txt files
data_names = [f for f in os.listdir(folder_path) if f[-3:]=='txt']
data_names[:5]

['BrotherInArms.txt',
 'DarkMirror.txt',
 'DestinyWar.txt',
 'DreamsofDestruction1.txt',
 'DreamsofDestruction10.txt']

In [3]:
'''
Getting txt contents


declare raw_texts
for each file
    extract file contents
    append to raw_texts

'''


raw_texts = []

for txt in data_names:
    with open(folder_path + txt, 'rb') as f:
        single_text = f.read().decode(encoding='utf-8')
    raw_texts.append(single_text.lower())
    
# Prints out the first 2 files    
raw_texts[:2]

['five years ago...\r\n\r\nmazeka dove aside even as the acid blade slashed through the air where he had been standing. he could hear the angry hiss of centuries-old rock dissolving where the sword had brushed against it. a step slower and that would have been his armor.\r\n\r\nhe hit the ground and rolled, ending up back on his feet with dagger at the ready. vultraz twirled the blade over his head, smiling. “you knew it had to come down to this, didn’t you?” said the crimson-armored matoran. “just the two of us, mask to mask.”\r\n\r\n“this isn’t one of your epic fables,” mazeka replied. “you’re a thief and a murderer, vultraz. you killed an entire village of matoran who never did a thing to you.”\r\n\r\n“except have something i wanted – an intact lava-gem, a rare find on the tren krom peninsula,” vultraz replied. “they didn’t want to give it up... thought it appeased the volcano or some such thing, kept it from erupting... a few well-timed explosions and one sea of lava later, and the

# 2. Data Preprocessing

To work with text data, you can work on the character level or the word level. 
Each have their own advantages

First, turn words (human interpretable) into integers (machine interpretable)  
if character, need to extact each unique character

if word, need to isolate each word (tokenize)

then turn each unique character/word into an integer

In [4]:
'''
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,/:;<=>?@[\\]^_`{|}~\t\n',
)

tokenizer.fit_on_texts(raw_texts)

vocab_size = len(tokenizer.word_index)

def word2int(texts):
    # tokenizes then converts to float
    int_tokens = tokenizer.texts_to_sequences(texts)
    tensor = tf.convert_to_tensor(int_tokens, dtype=float)
    
    # [1:7928] -> [-1:1]
    # (T-1 - 7927/2) / (7927/2)
    return shift(tensor)


def int2word(texts):
    # takes integer tokens into words
    unshifted = unshift(texts)
    rounded = tf.round(unshifted)
    int_tokens = tf.cast(rounded,dtype=tf.int32)
    return tokenizer.sequences_to_texts(int_tokens.numpy().tolist())

def tokenize(words):
    assert type(words) == list, "Expecting ['string1','string2', ..., 'stringn']"
    tokens = []
    for text in words: 
        

def tensor2list(tensor):
    return [tensor.numpy().tolist()]

def shift(array,n=vocab_size):
    
    return (array-1)/(n-1)

def unshift(array, n=vocab_size):
    return array*(n-1) + 1
'''


'\ntokenizer = tf.keras.preprocessing.text.Tokenizer(\n    filters=\'!"#$%&()*+,/:;<=>?@[\\]^_`{|}~\t\n\',\n)\n\ntokenizer.fit_on_texts(raw_texts)\n\nvocab_size = len(tokenizer.word_index)\n\ndef word2int(texts):\n    # tokenizes then converts to float\n    int_tokens = tokenizer.texts_to_sequences(texts)\n    tensor = tf.convert_to_tensor(int_tokens, dtype=float)\n    \n    # [1:7928] -> [-1:1]\n    # (T-1 - 7927/2) / (7927/2)\n    return shift(tensor)\n\n\ndef int2word(texts):\n    # takes integer tokens into words\n    unshifted = unshift(texts)\n    rounded = tf.round(unshifted)\n    int_tokens = tf.cast(rounded,dtype=tf.int32)\n    return tokenizer.sequences_to_texts(int_tokens.numpy().tolist())\n\ndef tokenize(words):\n    assert type(words) == list, "Expecting [\'string1\',\'string2\', ..., \'stringn\']"\n    tokens = []\n    for text in words: \n        \n\ndef tensor2list(tensor):\n    return [tensor.numpy().tolist()]\n\ndef shift(array,n=vocab_size):\n    \n    return (array-

In [5]:
tokens = nltk.tokenize.word_tokenize(" ".join(raw_texts))

vocab = sorted(set(tokens))
#vocab_size = len(vocab) #7792

w2i = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

i2w = tf.keras.layers.StringLookup(
    vocabulary=w2i.get_vocabulary(), invert=True, mask_token=None)

def tokenize(strings):
    return nltk.tokenize.word_tokenize(strings)

def tensor2string(tensor):
    return np.array2string(tensor.numpy())[3:-2]


vocab_size = len(w2i.get_vocabulary())

In [6]:
test = "Five years ago...Mazeka dove aside even as the acid blade slashed through the air where he had been standing"

tok = tokenize(test)
print(tok)
inties = w2i(tok)
print(inties)
floaties = inties/vocab_size
print(floaties)

['Five', 'years', 'ago', '...', 'Mazeka', 'dove', 'aside', 'even', 'as', 'the', 'acid', 'blade', 'slashed', 'through', 'the', 'air', 'where', 'he', 'had', 'been', 'standing']
tf.Tensor(
[   0 7747  198   34    0 1919  433 2189  427 6798  127  728 6080 6858
 6798  221 7551 3036 2941  633 6340], shape=(21,), dtype=int64)
tf.Tensor(
[0.         0.99409727 0.02540742 0.00436289 0.         0.24624663
 0.05556268 0.28089311 0.05479276 0.87232131 0.01629668 0.09341717
 0.78018735 0.88002053 0.87232131 0.02835878 0.96894649 0.38958039
 0.37738997 0.08122674 0.81355062], shape=(21,), dtype=float64)


In [7]:
next_char = tf.constant(['This was written by a bot \n'])

tokenize(tensor2string(next_char))

['This', 'was', 'written', 'by', 'a', 'bot', '\\n']

# 3. Making the dataset

In [8]:
int_dataset = tf.data.Dataset.from_tensor_slices(w2i(tokens)/vocab_size)


for ints in int_dataset.take(10):
    print(i2w(ints*vocab_size).numpy().decode('utf-8'))

five
years
ago
...
mazeka
dove
aside
even
as
the


In [9]:
seq_length = 75
# need to convert to integers and count elements to get length of text
examples_per_epoch = len(tokens)//(seq_length+1)

In [10]:
# split data into batches
sequences = int_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(i2w(seq*vocab_size))

tf.Tensor(
[b'five' b'years' b'ago' b'...' b'mazeka' b'dove' b'aside' b'even' b'as'
 b'the' b'achieving' b'blade' b'slashed' b'through' b'the' b'air' b'where'
 b'he' b'had' b'been' b'standing' b'-dead' b'he' b'could' b'hear' b'the'
 b'angry' b'hiss' b'of' b'centuries-old' b'rock' b'dissolving' b'where'
 b'the' b'sword' b'had' b'brushed' b'against' b'it' b'-dead' b'a' b'step'
 b'slower' b'and' b'that' b'would' b'have' b'been' b'his' b'armor'
 b'-dead' b'he' b'hit' b'the' b'ground' b'and' b'rolled' b',' b'ending'
 b'up' b'back' b'on' b'his' b'feet' b'with' b'dagger' b'at' b'the'
 b'ready' b'-dead' b'vultraz' b'twirled' b'the' b'blade' b'over' b'his'], shape=(76,), dtype=string)


In [11]:
# function to give previous n words, then predict the next word
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [12]:
print(split_input_target(list("Tensorflow")))
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", i2w(input_example*vocab_size).numpy())
    print("Target:", i2w(target_example*vocab_size).numpy())

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'], ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])
Input : [b'five' b'years' b'ago' b'...' b'mazeka' b'dove' b'aside' b'even' b'as'
 b'the' b'achieving' b'blade' b'slashed' b'through' b'the' b'air' b'where'
 b'he' b'had' b'been' b'standing' b'-dead' b'he' b'could' b'hear' b'the'
 b'angry' b'hiss' b'of' b'centuries-old' b'rock' b'dissolving' b'where'
 b'the' b'sword' b'had' b'brushed' b'against' b'it' b'-dead' b'a' b'step'
 b'slower' b'and' b'that' b'would' b'have' b'been' b'his' b'armor'
 b'-dead' b'he' b'hit' b'the' b'ground' b'and' b'rolled' b',' b'ending'
 b'up' b'back' b'on' b'his' b'feet' b'with' b'dagger' b'at' b'the'
 b'ready' b'-dead' b'vultraz' b'twirled' b'the' b'blade' b'over']
Target: [b'years' b'ago' b'...' b'mazeka' b'dove' b'aside' b'even' b'as' b'the'
 b'achieving' b'blade' b'slashed' b'through' b'the' b'air' b'where' b'he'
 b'had' b'been' b'standing' b'-dead' b'he' b'could' b'hear' b'the'
 b'angry' b'hiss' b'of' b'centuries-o

In [13]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

In [14]:
# shuffling the data
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

# 4. Modelling

In [15]:
# The embedding dimension
# the number of factors for the model to find
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
class RNN(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # embedding: vocabs -> embedding space
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # gru: 1 GRU layer
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # output layer
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    print(x.shape)
    if states is None:
      states = self.gru.get_initial_state(x)
    # input -> gru layer
    x, states = self.gru(x, initial_state=states, training=training)
    print(x.shape)
    # gru -> output layer
    x = self.dense(x, training=training)
    print(x.shape)

    if return_state:
      return x, states
    else:
      return x

In [17]:
model = RNN(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(w2i.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 75, 256)
(256, 75, 1024)
(256, 75, 7793)
(256, 75, 7793) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "rnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1995008   
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  7987825   
                                                                 
Total params: 13,921,137
Trainable params: 13,921,137
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()/vocab_size
sampled_indices

array([0.18555114, 0.36109329, 0.84011292, 0.4035673 , 0.67329655,
       0.33658411, 0.465546  , 0.18041832, 0.71628384, 0.59720262,
       0.04465546, 0.74040806, 0.21878609, 0.00410625, 0.79763891,
       0.10766072, 0.04029257, 0.37751829, 0.95868087, 0.24137046,
       0.37354036, 0.78044399, 0.89362248, 0.64288464, 0.17695368,
       0.48415244, 0.23790581, 0.60130887, 0.15154626, 0.68959322,
       0.11099705, 0.37469524, 0.98460157, 0.53252919, 0.91864494,
       0.74027974, 0.95803927, 0.03952265, 0.9645836 , 0.22302066,
       0.65225202, 0.77364301, 0.08263827, 0.49403311, 0.59040164,
       0.8678301 , 0.03721288, 0.40767355, 0.93789298, 0.95932247,
       0.21121519, 0.96124727, 0.62825613, 0.8842551 , 0.01809316,
       0.46323624, 0.03618632, 0.10239959, 0.3029642 , 0.14179392,
       0.46695753, 0.40805851, 0.10958553, 0.87963557, 0.58013602,
       0.1176697 , 0.20582574, 0.24355191, 0.05350956, 0.19722828,
       0.86693186, 0.43693058, 0.46464776, 0.00808418, 0.21557

In [21]:
print("Input:\n", i2w(input_example_batch[0]*vocab_size).numpy())
print()
print("Next Char Predictions:\n", i2w(sampled_indices*vocab_size).numpy())

Input:
 [b'they' b'have' b'long' b'memories.' b'\xe2\x80\x9d' b'it' b'only'
 b'took' b'a' b'few' b'more' b'minutes' b'of' b'whispered' b'conversation'
 b'before' b'spiriah' b'turned' b'back' b'to' b'the' b'group' b'and'
 b'announced' b'that' b'he' b'would' b'be' b'their' b'guide' b'to'
 b'artidax' b'-dead' b'he' b'immediately' b'set' b'off' b'inland' b','
 b'with' b'the' b'rest' b'following' b'-dead' b'lariska' b'fell' b'in'
 b'beside' b'brutaka' b',' b'saying' b',' b'\xe2\x80\x9c' b'you' b'know'
 b'full' b'well' b'he' b'could' b'never' b'be' b'leader' b'of' b'the'
 b'brotherhood.' b'\xe2\x80\x9d' b'\xe2\x80\x9c' b'let' b'him' b'think'
 b'he' b'might' b'get' b'to' b'be']

Next Char Predictions:
 [b'cracked' b'going' b'summoning' b'hold' b'readers' b'fragile' b'knock'
 b'corrupt' b'rodent-like.' b'pack' b'apart' b'scurrying' b'described'
 b'-dead' b'southern' b'boy' b'ankles' b'hadn' b'water-breather' b'dodged'
 b'guarantee' b'slaughter' b'traced' b'primitive' b'conversation'
 b'legion'

In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)
print(tf.exp(mean_loss).numpy(),vocab_size)

Prediction shape:  (256, 75, 7793)  # (batch_size, sequence_length, vocab_size)
Mean loss:         8.951548
7719.8306 7793


## Generating 1 step of the model

In [24]:
'''
OneStep pseudocode

get model trained from data
get words to float and float to words dictionary

def generate one step
    tokenize inputs
    get float representation saved into a tensor
    
    feed input-floats into model to get predictions
    
    use only last prediction
    use prediction mask to remove [UNK]
    
    randomly sample from the predictions
    turn prediction-float into word
    
    return word and state


'''

'\nOneStep pseudocode\n\nget model trained from data\nget words to float and float to words dictionary\n\ndef generate one step\n    tokenize inputs\n    get float representation saved into a tensor\n    \n    feed input-floats into model to get predictions\n    \n    use only last prediction\n    use prediction mask to remove [UNK]\n    \n    randomly sample from the predictions\n    turn prediction-float into word\n    \n    return word and state\n\n\n'

In [25]:

# already tokenized and in float representation
for test_in, test_out in dataset.take(1):
    print("test input:\n",i2w(test_in*vocab_size), 
          "\ntest output:\n",i2w(test_out*vocab_size))

predicted_logits, states = model(inputs=test_in, states=None,
                                          return_state=True)
print(predicted_logits.shape)
print("\npredicted output:\n",predicted_logits)

test input:
 tf.Tensor(
[[b'if' b'they' b'stayed' ... b'and' b'two' b'dark']
 [b'over' b'there' b'were' ... b'toa' b'of' b'fire']
 [b'plant' b'life' b'-dead' ... b'about' b'him' b'-dead']
 ...
 [b'time' b'-dead' b"'" ... b'gone' b',' b'he']
 [b'was' b'the' b'skopio' ... b'they' b'sabotaged' b'it']
 [b'whispered' b'-dead' b'suddenly' ... b'the' b'world' b'ended']], shape=(256, 75), dtype=string) 
test output:
 tf.Tensor(
[[b'they' b'stayed' b'together' ... b'two' b'dark' b'hunters']
 [b'there' b'were' b'matoran' ... b'of' b'fire' b'looked']
 [b'life' b'-dead' b'one' ... b'him' b'-dead' b'a']
 ...
 [b'-dead' b"'" b'``' ... b',' b'he' b'said']
 [b'the' b'skopio' b'vehicle' ... b'sabotaged' b'it' b'-dead']
 [b'-dead' b'suddenly' b',' ... b'world' b'ended' b'-dead']], shape=(256, 75), dtype=string)
(256, 75, 256)
(256, 75, 1024)
(256, 75, 7793)

predicted output:
 tf.Tensor(
[[[ 3.9585037e-03 -1.4731116e-04  2.5045411e-03 ... -3.6797617e-04
    3.6991688e-03 -5.9036165e-03]
  [ 6.4009549e-0

In [26]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    # input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    # input_ids = self.ids_from_chars(input_chars).to_tensor()
    input_chars = tokenize(inputs)
    input_ids = self.ids_from_chars(input_chars) #iput_id = integer tensor

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids/vocab_size, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids*vocab_size)

    # Return the characters and model state.
    return predicted_chars, states

# 5. Training

In [27]:
attempt = 3
EPOCHS = 100

In [28]:
'''
model = RNN(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
'''

'\nmodel = RNN(\n    # Be sure the vocabulary size matches the `StringLookup` layers.\n    vocab_size=vocab_size,\n    embedding_dim=embedding_dim,\n    rnn_units=rnn_units)\n'

In [29]:
model.compile(optimizer='adam', loss=loss)

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = f'Attempt{attempt}/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
(256, 75, 256)
(256, 75, 1024)
(256, 75, 7793)
(256, 75, 256)
(256, 75, 1024)
(256, 75, 7793)
7/7 [==============================] - 3s 290ms/step - loss: 4.9567
Epoch 2/100
7/7 [==============================] - 2s 294ms/step - loss: 0.2532
Epoch 3/100
7/7 [==============================] - 2s 288ms/step - loss: 0.1496
Epoch 4/100
7/7 [==============================] - 2s 291ms/step - loss: 0.1066
Epoch 5/100
7/7 [==============================] - 2s 288ms/step - loss: 0.0874
Epoch 6/100
7/7 [==============================] - 2s 292ms/step - loss: 0.0810
Epoch 7/100
7/7 [==============================] - 2s 286ms/step - loss: 0.0766
Epoch 8/100
7/7 [==============================] - 2s 292ms/step - loss: 0.0726
Epoch 9/100
7/7 [==============================] - 2s 284ms/step - loss: 0.0683
Epoch 10/100
7/7 [==============================] - 2s 289ms/step - loss: 0.0635
Epoch 11/100
7/7 [==============================] - 2s 292ms/step - loss: 0.0577
Epoch 12/100
7/7 [======

In [32]:
one_step_model = OneStep(model, i2w, w2i)

In [33]:
model.save(f'Attempt{attempt}/final_model')

(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)


INFO:tensorflow:Assets written to: Attempt3/final_model\assets


INFO:tensorflow:Assets written to: Attempt3/final_model\assets


In [34]:
tf.saved_model.save(one_step_model, f'Attempt{attempt}/one_step')

(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)
(None, 75, 256)
(None, 75, 1024)
(None, 75, 7793)


INFO:tensorflow:Assets written to: Attempt3/one_step\assets


INFO:tensorflow:Assets written to: Attempt3/one_step\assets


Sample generation

In [35]:
start = time.time()
states = None
next_char = 'This was written by a bot\n'
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(" "+next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

(6, 256)


ValueError: in user code:

    File "<ipython-input-26-3a4807a767d7>", line 29, in generate_one_step  *
        predicted_logits, states = self.model(inputs=input_ids/vocab_size, states=states,
    File "C:\Users\nthnt\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "rnn" (type RNN).
    
    in user code:
    
        File "<ipython-input-16-3eea4c5500fc>", line 20, in call  *
            x, states = self.gru(x, initial_state=states, training=training)
        File "C:\Users\nthnt\anaconda3\lib\site-packages\keras\layers\recurrent.py", line 737, in __call__  **
            return super(RNN, self).__call__(inputs, **kwargs)
        File "C:\Users\nthnt\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
            raise e.with_traceback(filtered_tb) from None
        File "C:\Users\nthnt\anaconda3\

In [ ]:
f= open(f"Attempt{attempt}/Generated_Text.txt","w+")
f.write(result[0].numpy().decode('utf-8'))
f.close()

# 6. Generating Data

In [ ]:
attempt = 3
checkpoint_dir = f'Attempt{attempt}/training_checkpoints/'

In [ ]:
# grab weights from last trained model
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = RNN(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.load_weights(latest)

In [ ]:
one_step_model = OneStep(model, i2w, w2i)

In [ ]:
states = None
next_char = tf.constant(['Attempt1:\n','Attempt2:\n','Attempt3:\n','Attempt4:\n']) # seed
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(" "+next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode("utf-8"))

In [ ]:
# saving generated data
for i in range(len(result)):
    f= open(f"Attempt{attempt}/Generated_Text_{i}.txt","w+")
    f.write(result[i].numpy().decode("utf-8"))
    f.close()